In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset_path = '/home/laptop-kl-11/personal_project/car_dekho_project/dataset/extracted_dataset/extracted_data.csv'
df = pd.read_csv(dataset_path)

In [3]:
pd.set_option('display.max_columns', None)

In [15]:
#Manually going through each column and writing the TODO down:
change_column_names = {'ft':'fuel_type','bt': 'body_type','km': 'kilo_meter',
                       'ownerNo': 'no_of_previous_owner',}
'''
a) drop "it" column because it has only 0 ------------------------->"Done"
b) OHE "ft" column and it is data imbalance ----------------------->"Done"
c) OHE "bt" column and it is data imbalance ----------------------->"Done"
d) convert "km" to numerical data and normalize ------------------->"Done"
e) OHE or label encode the "transmission" ------------------------->"Done"
f) "ownerNo" doesn't need much preprocessing, may be try normalize 
g) drop "owner" ---------------------------------------------------> "Done"
h) drop "oem"   ---------------------------------------------------> "Done"
i) OHE "model"   
j) normalize modelYear   ------------------------------------------> "Done"
k) drop "centralVariantId" ----------------------------------------> "Done"
l) drop "variantName"  --------------------------------------------> "Done"
m) preprocess "price" - remember it the target variable 
n) drop "priceActual","priceSaving","priceFixedText", trendingText --> "Done"
o) OHE "location" ---------------------------------------------------> "Done"
p) Extract only year from "Registration Year" and if need drop the modelYear --------------------->"Done"
q) Check "Insurance Validity" is correlated with Price and if need Fill the missing values -------> "Done"
r) Drop the "Fuel Type" column -------------------------------------------------------------------> "Done"
s) Convert the "Seats", "Kms Driven" into numerical column and handle missing values ---------------> "Done"
t) Check "RTO" has impact with the target column --------------------------------------------------->'Done"
u) drop "Ownership" --------------------------------------------------------------------------------> "Done"
v) handle "Engine Displacement" missing values and convert to numerical data ------------------------> "Done"
w) drop "Transmission", "Year of Manufacture","Values per Cylinder","Value Configuration" -----------> "Done"
ww) drop "BoreX Stroke",'Compression Ratio',"Max Power", "Max Torque","Super Charger" --------------> "Done"
x) Handle repetition "Turbo Charger", 'No of Cylinder' ------------------> "Done"
y) drop "Cargo Volumn"  -------------------------------------------------> "Done"
z) Handle repetition "Rear Brakae Type", "Drive Type" --------------------> "Done"
aa) Handle "No Door Numbers", "Front Brake Type" -------------------------> "Done"
bb) Handle repetition "Steering Type" -----------------------------------> "Done"
cc) drop "Top Speed", "Acceleration", "Turning Radius","Alloy Wheel Size"--> "Done"
dd) Handle "Tyre Type", "Gear Box", "Seating Capacity" ------------------> "Done"
ee) Handle "Kerb Weight" ------------------------------------------------> "Done"
ff) drop "Rear Tread"  --------------------------------------------------> "Done"
gg) handle "Height", "Wheel Base" ---------------------------------------> "Done"
hh) drop "Gross Weight" -------------------------------------------------> "Done"
ii) Handle "Length", "Width" --------------------------------------------> "Done"
jj) drop "Front Tread", "Ground Clearance Unladen"  ---------------------> "Done"
kk) Handle "Displacement" -----------------------------------------------> "Done"
ll) Drop "Engine Type" --------------------------------------------------> "Done"

color, price, model, original_equipment_manufacturer



Skewness in the data
fuel_type
body_type
ownerNo

Normalize: 
 kilometer
 OwnerNo
 modelYear
 Length , Width, Height, Wheel_base


required_features: 
'location' - dropdown option
'body_type' - dropdown option
'fuel_type' - dropdown 




'''

'\na) drop "it" column because it has only 0 ------------------------->"Done"\nb) OHE "ft" column and it is data imbalance ----------------------->"Done"\nc) OHE "bt" column and it is data imbalance ----------------------->"Done"\nd) convert "km" to numerical data and normalize ------------------->"Done"\ne) OHE or label encode the "transmission" ------------------------->"Done"\nf) "ownerNo" doesn\'t need much preprocessing, may be try normalize \ng) drop "owner" ---------------------------------------------------> "Done"\nh) drop "oem"   ---------------------------------------------------> "Done"\ni) OHE "model"   \nj) normalize modelYear   ------------------------------------------> "Done"\nk) drop "centralVariantId" ----------------------------------------> "Done"\nl) drop "variantName"  --------------------------------------------> "Done"\nm) preprocess "price" - remember it the target variable \nn) drop "priceActual","priceSaving","priceFixedText", trendingText --> "Done"\no) OHE

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
from typing import List

In [17]:
#preprocessing price feature
def preprocess_values(x): 
        value = ''.join(char for char in x if char.isdigit() or char=='.')
        if (value.count('0') >= 3) and ('.' not in value): 
            return 100000 - float(value)
        else: 
            if '.' in value: #for values like 7.62, 53.3
                return (float(value.split('.')[0])*100000) + (float(value.split('.')[1])*1000)
            else: 
                return float(value) * 100000
            
df['price'] = df['price'].apply(preprocess_values)

In [18]:
class RemoveUnwantedColsAndRows(BaseEstimator,TransformerMixin):
    def __init__(self, unwanted_columns: List[str], non_nan_row_threshold:int = 90):
        self.unwanted_columns = unwanted_columns
        self.non_nan_row_threshold = non_nan_row_threshold
        return None

    def fit(self,X):
        self.column = X.columns
        return self

    def transform(self,X):
        x_transformed = X.copy()
        x_transformed.drop(self.unwanted_columns,axis=1,inplace = True)
        #removing rows which has Nan more than given threshold
        x_transformed.dropna(thresh = self.non_nan_row_threshold, axis = 0, inplace = True)
        print("The dropped columns are ")
        print(' '.join([col for col in self.column if col not in x_transformed.columns]))
        return x_transformed.reset_index(drop=True)
    

class StandardizeColumnName(BaseEstimator, TransformerMixin):
    def __init__(self,rename_columns):
        self.rename_columns = rename_columns

    def fit(self, X):
        return self
    
    def transform(self, X):
        transformed_df = X.copy()
        transformed_df.rename(columns = self.rename_columns, inplace= True)
        transformed_df.columns = [col.lower().replace(' ','_') for col in transformed_df.columns]
        return transformed_df

    
class CustomStandardScaler(BaseEstimator, TransformerMixin): 
    def __init__(self,is_train): 
        self.is_train = is_train
        self.column_names = ['kerb_weight', 'height', 'wheel_base','length','width','kilo_meter',
                'model_year','engine_displacement']
        
    def fit(self, X): 
        self.x_ss = StandardScaler()
        self.x_ss.fit(X[(X[self.column_names]!=-1).any(axis=1)][self.column_names])
        self.y_ss = StandardScaler()
        self.y_ss.fit(X[['price']])
        return self
    
    def transform(self, X): 
        transformed_df = X.copy()
        output_df = pd.DataFrame(self.x_ss.transform(transformed_df[self.column_names]),
                                columns = self.x_ss.get_feature_names_out())
        if self.is_train:
            price_col = pd.DataFrame(self.y_ss.transform(transformed_df[['price']]),columns=['price'])
            transformed_df = transformed_df.drop('price',axis = 1)
            output_df = pd.concat([output_df,price_col],axis= 1)
        transformed_df = transformed_df.drop(self.column_names, axis= 1)
        transformed_df = pd.concat([transformed_df,output_df],axis=1)
        return transformed_df
    

class preprocess_numerical_column(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.column_names = ['length','width','height','wheel_base','kerb_weight','kilo_meter'] 
        pass

    def convert_numerical(self, x):
        #works for Length and Width features only
        if pd.notna(x):
            val  = str(x)
            if ','  in val:
                val = val.replace(',','')
            if '-' in val:
                val = val.split('-')[0]
            val = val.strip() 
            val = ''.join([char for char in val if char.isdigit()])
            if len(str(val))>1 :
                return int(val)
            else: 
                return None
        else:
            return None
    
    def convert_numerical_engine_cc(self, x):
        if pd.notna(x):
            val  = str(x).strip()
            val = ''.join([char for char in val if char.isdigit()])
            if int(val)>600:
                return int(val)
            else: 
                
                return None
        else:
            return None
    
    def fit(self, X):
        return self
    

    def transform(self, X):
        transformed_df = X.copy()
        #preprocess values
        for col in self.column_names:
            transformed_df[col] = transformed_df[col].apply(self.convert_numerical)
        transformed_df['engine_displacement'] = transformed_df['engine_displacement'].apply(self.convert_numerical_engine_cc)

        return transformed_df
    


class impute_numerical_missing_values(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.col_names = ['length','width','wheel_base','height','kerb_weight','engine_displacement','kilo_meter']
        

    def impute_null_with_lookup(self,model_name, model_year, look_up_table):
        if (model_name,model_year) in look_up_table.index:
            return look_up_table.loc[(model_name,model_year)]
        elif model_name in [model_name for model_name,_ in look_up_table.index]: 
            return look_up_table.loc[(model_name)].iloc[-1]
        else:
            return -1 

    def fit(self, X):
        X_copy = X.copy()
        self.lookup_tables = {}
        for col in self.col_names: 
            self.lookup_tables[col] = X_copy[pd.notna(X_copy[col])].groupby(['model','model_year'])[col].mean()
        return self
    
    def transform(self, X):
        transformed_df = X.copy()        

        for row in X[X.isna().any(axis=1)].iterrows():
            model_name = row[1]['model']
            model_year = row[1]['model_year']
            for col in self.col_names:
                val = row[1][col]
                if pd.isna(val):
                    impute_value = self.impute_null_with_lookup(model_name, model_year, self.lookup_tables[col])
                    transformed_df.iloc[row[0],list(X.columns).index(col)] = impute_value
    
        return transformed_df

    


class preprocess_categorical_features(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass 
    

    def turbo_charger_encode(self,x): 
        if pd.isna(x): 
            return None
        else:
            lower_x = x.lower().strip()
            if ('yes' in lower_x) or ('twin' in lower_x) or ('turbo' in lower_x):
                return 1
            else: 
                return 0
            

    def brake_encode(self,x): 
        if pd.isna(x):
            return None
        else :
            lower_x = x.lower().strip()
            if ('disc' in lower_x) or ('abs' in lower_x) or ('disk' in lower_x):
                return 1
            elif ('drum' in lower_x):
                return 0
            else:
                return None
                
    def tyre_preprocessing(self,x): 
        if pd.isna(x):
            return 'None/other'
        else:
            lower_x = x.lower().strip()
            
            if ('radial' in lower_x ) and ('runflat' in lower_x):
                return 'radial_runflat'
            elif ('runflat' in lower_x) and ('tubeless' in lower_x):
                return 'runflat_tubeless'
            elif ('tubeless' in lower_x) and ('radial' in lower_x):
                return 'tubeless_radial'
            elif 'runflat' in lower_x:
                return 'runflat'
            elif 'radial' in lower_x: 
                return 'radial'
            elif 'tubeles' in lower_x: 
                return 'tubeless'
            else: 
                return 'None/other'
            
    def transmission_encode(self,x): 
        x = x.lower().strip()
        if 'automatic' == x:
            return 0
        else: 
            return 1
        
    def cylinder_encode(self,x):
        if pd.notna(x):
            if x > 8:
                return None
            else: 
                return x
        else:
            return x


    def fit(self, X): 
        return self
    
    def transform(self, X):
        transformed_x = X.copy()
        transformed_x['front_brake_type'] = transformed_x['front_brake_type'].apply(self.brake_encode)
        transformed_x['rear_brake_type'] = transformed_x['rear_brake_type'].apply(self.brake_encode)
        transformed_x['turbo_charger']  = transformed_x['turbo_charger'].apply(self.turbo_charger_encode)
        transformed_x['transmission'] = transformed_x['transmission'].apply(self.transmission_encode)
        transformed_x['no_of_cylinder'] = transformed_x['no_of_cylinder'].apply(self.cylinder_encode)
        transformed_x['tyre_type'] = transformed_x['tyre_type'].apply(self.tyre_preprocessing)
        return transformed_x                    

        
class impute_categorical_feature(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.column_names = ['front_brake_type','rear_brake_type','turbo_charger','transmission','no_of_cylinder','tyre_type']
        
    def impute_null_with_lookup(self,model_name, model_year, col_name):
        look_up_table = self.lookup_table[col_name]
        if (model_name,model_year) in look_up_table.index:
            val = look_up_table.loc[(model_name,model_year)]
            return val
        elif model_name in [model_name for model_name,_ in look_up_table.index]: 
            val = look_up_table.loc[(model_name)].iloc[-1]
            return val
        else:
            return -1 

    def fit(self, X):
        self.type = []
        def get_mode(val): 
            mode = pd.Series.mode(val)
            self.type.append(type(mode))
            if len(mode)>1:
                return mode[0]
            else: 
                return mode
            
        X_copy = X.copy()
        self.lookup_table = {
            col: X_copy[pd.notna(X_copy[col])].groupby(['model','model_year'])[col].agg(get_mode) for col in self.column_names}
        return self
        
    def transform(self,X):
        transformed_df = X.copy()
        for row in X[X.isna().any(axis=1)].iterrows():
            model_name = row[1]['model']
            model_year = row[1]['model_year']
            
            for col in self.column_names:
                val = row[1][col]

                if pd.isna(val):
                    impute_value = self.impute_null_with_lookup(model_name, model_year,col)
                    transformed_df.iloc[row[0],list(X.columns).index(col)] = impute_value
    
        return transformed_df
    

    
class OneHotEncoding(BaseEstimator, TransformerMixin):
    def __init__(self,column_names): 
        self.column_names = column_names

    def fit(self, X):
            self.ohe = OneHotEncoder(sparse_output=False,dtype = int)
            self.ohe.fit(X[self.column_names])
            return self
    
    def transform(self,X): 
            ohe_value = self.ohe.transform(X[self.column_names])
            tranformed_df = pd.DataFrame(ohe_value, columns=self.ohe.get_feature_names_out())
            tranformed_df = pd.concat([X,tranformed_df], axis= 1)
            return tranformed_df.drop(self.column_names,axis = 1)
    

class TargetEncoding(BaseEstimator, TransformerMixin): 
    def __init__(self): 
        self.column_names = ['original_equipment_manufacturer', 'model', 'color']
        pass
        
    def fit(self, X):
        #target encoding
        self.mean_encoding = {}
        for col in self.column_names: 
            groupby_results = X[[col,'price']].groupby([col]).agg(mean_price = ('price','mean'),
                                                                          count = ('price','size'))
            
            self.mean_encoding[col] = groupby_results[groupby_results['count']>5]['mean_price']
        return self

    def transform(self, X):
        transformed_df = X.copy()
        col_idxes = [(list(X.columns).index(col),col) for col in self.column_names]
        for idx in range(X.shape[0]):
            for col_idx,col in col_idxes:
                val = transformed_df.iloc[idx,col_idx] 
                if pd.notna(val) and (val in self.mean_encoding[col]):
                    transformed_df.iloc[idx,col_idx] = self.mean_encoding[col][val]
                else:
                    transformed_df.iloc[idx,col_idx] = -1

        return transformed_df




In [19]:

numerical_pipeline = Pipeline([
    ('remove_unwanted_col_row', RemoveUnwantedColsAndRows(
        unwanted_columns = ["it","owner","centralVariantId","variantName","priceActual","priceSaving","priceFixedText",
                            "trendingText","Fuel Type","Ownership","Transmission","Year of Manufacture","Values per Cylinder",
                            "Value Configuration","BoreX Stroke","Compression Ratio","Max Power","Max Torque","Super Charger","Cargo Volumn","Top Speed",
                            "Acceleration","Turning Radius","Alloy Wheel Size","Rear Tread","Gross Weight","Front Tread","Engine Type","Ground Clearance Unladen",
                            "Seating Capacity","Gear Box","Steering Type", "Drive Type","No Door Numbers",'Displacement','RTO','Seats','Kms Driven','Registration Year',
                            'Insurance Validity','Fuel Suppy System','No Of Airbags','Number Of Speaker'],
        non_nan_row_threshold = 90)),

    ('standardize_column_names', StandardizeColumnName(
        
        rename_columns={'ft':'fuel_type','bt': 'body_type','km': 'kilo_meter', 'oem': 'original_equipment_manufacturer','modelYear': 'model_year',
                        'Wheel Base': 'wheel_base','Kerb Weight': 'kerb_weight','Rear Brake Type': 'rear_brake_type','Front Brake Type': 'front_brake_type',
                        'No of Cylinder': 'no_of_cylinder','Tyre Type' : 'tyre_type','Turbo Charger': 'turbo_charger'})),

    ('formatting_column', preprocess_numerical_column()),
    ('impute_missing_values',impute_numerical_missing_values()),
    ('standardization', CustomStandardScaler(is_train=True))
    
])



categorical_pipeline = Pipeline([
    ('formatting_columns',preprocess_categorical_features()),
    ('impute_missing_values',impute_categorical_feature()),
    ('ohe', OneHotEncoding(column_names = ['fuel_type', 'body_type','location','no_of_cylinder','tyre_type'])),
    ('target_encoding', TargetEncoding())
])

In [20]:
final_pipeline = Pipeline([
    ('numerical_pipeline', numerical_pipeline),
    ('categorical_pipeline',categorical_pipeline),
    ])

In [21]:
preprocessed_df = final_pipeline.fit_transform(df)

The dropped columns are 
it owner centralVariantId variantName priceActual priceSaving priceFixedText trendingText Registration Year Insurance Validity Fuel Type Seats Kms Driven RTO Ownership Transmission Year of Manufacture No Of Airbags Number Of Speaker Values per Cylinder Value Configuration BoreX Stroke Compression Ratio Max Power Max Torque Super Charger Fuel Suppy System Engine Type Displacement Ground Clearance Unladen Front Tread Gross Weight Rear Tread Seating Capacity Gear Box Alloy Wheel Size Turning Radius Acceleration Top Speed Steering Type No Door Numbers Cargo Volumn Drive Type


In [22]:
preprocessed_df

(8369, 226)

In [23]:
preprocessed_df.to_csv('../dataset/preprocessed_data/preprocessed_df.csv')

In [32]:
list(df.columns)

['it',
 'ft',
 'bt',
 'km',
 'transmission',
 'ownerNo',
 'owner',
 'oem',
 'model',
 'modelYear',
 'centralVariantId',
 'variantName',
 'price',
 'priceActual',
 'priceSaving',
 'priceFixedText',
 'trendingText',
 'location',
 'Registration Year',
 'Insurance Validity',
 'Fuel Type',
 'Seats',
 'Kms Driven',
 'RTO',
 'Ownership',
 'Engine Displacement',
 'Transmission',
 'Year of Manufacture',
 'Drive Modes',
 'Voice Control',
 'Smart Key Band',
 'Battery Saver',
 'Rear Reading Lamp',
 'Power Boot',
 'Power Steering',
 'Real Time Vehicle Tracking',
 'Luggage Hook And Net',
 'Cruise Control',
 'Seat Lumbar Support',
 'Power Windows Front',
 'Vanity Mirror',
 'Gear Shift Indicator',
 'Trunk Light',
 'Adjustable Headrest',
 'Engine Start Stop Button',
 'Find My Car Location',
 'Multifunction Steering Wheel',
 'Remote Engine Start Stop',
 'Active Noise Cancellation',
 'Remote Fuel Lid Opener',
 'Hands Free Tailgate',
 'Remote Trunk Opener',
 'Lane Change Indicator',
 'Glove Box Cooling',
